In [ ]:
# Memory-Augmented Neural Network Demo
This notebook demonstrates the functionality of a memory-augmented neural network that can store and retrieve information from a memory bank, implement attention-based memory access, calculate confidence scores for retrievals, and demonstrate learning over sequential data.


In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("C:/Users/sophy/graphfusion-challenge")
from model.memory import MemoryBank, ConfidenceScoring
from data.data_generator import generate_sequence_data

In [2]:
# Hyperparameters
seq_length = 10        # Length of sequences
feature_dim = 8        # Dimension of features
batch_size = 16        # Number of sequences in each batch
memory_size = 32       # Number of memory slots
learning_rate = 0.001  # Learning rate
num_epochs = 5         # Number of training epochs


In [3]:
# Generate sequential data
input_sequences, target_sequences = generate_sequence_data(seq_length, feature_dim, batch_size)

print("Input Sequences Shape:", input_sequences.shape)
print("Target Sequences Shape:", target_sequences.shape)


Input Sequences Shape: torch.Size([16, 10, 8])
Target Sequences Shape: torch.Size([16, 10, 8])


In [4]:
# Initialize Memory Bank and Confidence Scoring
memory_bank = MemoryBank(memory_size=memory_size, feature_dim=feature_dim)
confidence_scorer = ConfidenceScoring(feature_dim)
# After initializing MemoryBank and ConfidenceScoring
optimizer = torch.optim.Adam(
    list(memory_bank.parameters()) + list(confidence_scorer.parameters()),
    lr=learning_rate
)



In [5]:
# Define a simple Mean Squared Error loss function
loss_fn = torch.nn.MSELoss()

# Training loop
for epoch in range(num_epochs):
    for i in range(batch_size):
        # Extract the appropriate input sequence for the current index
        input_sequence = input_sequences[i]   
        
        # Reshape input_sequence to match the expected input data shape (1, feature_dim)
        input_sequence = input_sequence[-1].unsqueeze(0)   
        
        # Write to memory
        write_weights = torch.ones(1, memory_size) / memory_size  
        memory_bank.write(input_sequence, write_weights)
        
        # Retrieve data from memory
        retrieved_memory, _ = memory_bank.read(input_sequence)
        
        # Calculate loss
        loss = loss_fn(retrieved_memory, target_sequences[i])
        
        # Backpropagation (assuming optimizer is defined elsewhere)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}")


Epoch 1/5, Loss: 0.1893
Epoch 2/5, Loss: 0.0959
Epoch 3/5, Loss: 0.1119
Epoch 4/5, Loss: 0.2241
Epoch 5/5, Loss: 0.4205


C:\Users\sophy\anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([10, 8])) that is different to the input size (torch.Size([1, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [6]:
# Example of storing and retrieving data
test_input = input_sequences[0]  

# Print shape for debugging
print("Original Test Input Shape:", test_input.shape)

# Ensure write_weights has the correct shape for batch processing
write_weights = (torch.ones(test_input.size(0), memory_size) / memory_size)  

# Write to memory
memory_bank.write(test_input, write_weights)  
retrieved_memory, _ = memory_bank.read(test_input)  

print("Test Input:", test_input)
print("Retrieved Memory:", retrieved_memory)


Original Test Input Shape: torch.Size([10, 8])
Test Input: tensor([[0.6195, 0.4596, 0.3105, 0.3422, 0.8699, 0.7985, 0.2586, 0.8816],
        [0.1945, 0.0422, 0.3566, 0.0825, 0.0438, 0.2678, 0.6313, 0.0019],
        [0.4989, 0.4873, 0.2991, 0.2873, 0.9318, 0.5958, 0.7404, 0.0601],
        [0.5900, 0.5150, 0.4085, 0.5759, 0.4721, 0.9649, 0.2398, 0.2061],
        [0.6425, 0.4342, 0.5276, 0.9592, 0.4375, 0.1035, 0.0953, 0.3925],
        [0.4838, 0.6044, 0.6951, 0.3583, 0.0843, 0.2950, 0.0408, 0.4500],
        [0.7205, 0.5129, 0.9259, 0.4637, 0.4658, 0.1977, 0.1075, 0.9616],
        [0.5689, 0.1191, 0.1448, 0.0772, 0.8765, 0.7979, 0.4256, 0.0724],
        [0.7749, 0.0956, 0.8842, 0.9085, 0.9178, 0.0355, 0.7500, 0.0646],
        [0.4246, 0.6359, 0.0322, 0.7415, 0.0726, 0.6327, 0.4854, 0.0581]])
Retrieved Memory: tensor([[1.5714, 1.3421, 1.4615, 1.0227, 1.1192, 1.2224, 1.3019, 0.9800],
        [1.5714, 1.3421, 1.4615, 1.0227, 1.1192, 1.2224, 1.3019, 0.9800],
        [1.5714, 1.3421, 1.4615, 1

In [10]:
 # Define a simple Mean Squared Error loss function
loss_fn = torch.nn.MSELoss()

# Initialize an empty list to store loss values for each epoch
loss_values = []

# Training loop
for epoch in range(num_epochs):
    epoch_loss = 0  # Track total loss for the epoch
    
    for i in range(batch_size):
        # Extract the appropriate input sequence for the current index
        input_sequence = input_sequences[i]   
        
        # Reshape input_sequence to match the expected input data shape (1, feature_dim)
        input_sequence = input_sequence[-1].unsqueeze(0)   
        
        # Write to memory
        write_weights = torch.ones(1, memory_size) / memory_size  # Shape (1, memory_size)
        memory_bank.write(input_sequence, write_weights)
        
        # Retrieve data from memory
        retrieved_memory, _ = memory_bank.read(input_sequence)
        
        # Calculate loss
        loss = loss_fn(retrieved_memory, target_sequences[i])
        epoch_loss += loss.item()  # Accumulate loss
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Calculate average loss for the epoch
    avg_loss = epoch_loss / batch_size
    loss_values.append(avg_loss)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")

# Print the loss values for each epoch
print("\nTraining Loss per Epoch:")
for epoch, loss in enumerate(loss_values, 1):
    print(f"Epoch {epoch}: Loss = {loss:.4f}")


Epoch 1/5, Loss: 7.9915
Epoch 2/5, Loss: 9.1470
Epoch 3/5, Loss: 10.3815
Epoch 4/5, Loss: 11.6952
Epoch 5/5, Loss: 13.0880

Training Loss per Epoch:
Epoch 1: Loss = 7.9915
Epoch 2: Loss = 9.1470
Epoch 3: Loss = 10.3815
Epoch 4: Loss = 11.6952
Epoch 5: Loss = 13.0880
